In [ ]:
import ipyaladin.aladin_widget as ipyal
aladin = ipyal.Aladin(target='Berkeley 20', fov=0.3, survey='P/SDSS9/color')
aladin

In [ ]:
import bokeh
from bokeh.io import output_notebook, show, push_notebook
output_notebook(bokeh.resources.INLINE)

from hr.visual import _telescope_pointing_widget
from hr.visual import *

def hr_diagram_skyimage(cluster_name, output):
    """
    """
    cluster = get_hr_data(cluster_name)
    input_caption = 'Type in the name of your cluster and press Enter/Return:'
    text_input = TextInput(value=cluster.name, title=input_caption)
    pf = hr_diagram_figure(cluster)
    pf_image = skyimage_figure(cluster)
    layout = column(text_input, _telescope_pointing_widget(cluster.name),
                    pf, sizing_mode="scale_width")
    with output:
      h = show(layout, notebook_handle=True)
      return h

outw = widgets.Output()

q1 = widgets.VBox(children=[widgets.Label("1.	What are the ranges of star temperatures and luminosities that you observed on your H-R diagram?"), widgets.Textarea(rows=1)])
q1 = widgets.VBox(children=[widgets.Label("1.	What are the ranges of star temperatures and luminosities that you observed on your H-R diagram?"), widgets.Textarea(rows=1)])
q2 = widgets.VBox(children=[widgets.Label("2.	Where on the H-R diagram are the stars most concentrated? What is their color?"), widgets.Textarea(rows=1)])
q3 = widgets.VBox(children=[widgets.Label("3.	Where on the H-R diagram are cool and dim stars located?"), widgets.Textarea(rows=1)])

# display widget!!!!
display(widgets.HBox(children=[outw, widgets.VBox([q1, q2, q3])]))

h_comm = hr_diagram_skyimage("berkeley20", outw)

In [ ]:

from bokeh.io import output_notebook, show, push_notebook
output_notebook(bokeh.resources.INLINE)
from bokeh.models import ColumnDataSource, Circle
import ipywidgets as widgets
from IPython.display import display

plot = figure()
source = ColumnDataSource(
    data=dict(
        x=[1],
        y=[1],
    )
)

circle = Circle(x="x", y="y", size=15, fill_color="blue", fill_alpha=0.8, line_color=None)
plot.add_glyph(source, circle)

q1 = widgets.VBox(children=[widgets.Label("1.	What are the ranges of star temperatures and luminosities that you observed on your H-R diagram?"), widgets.Textarea(rows=1)])

outw = widgets.Output()

# display widget!!!!
display(widgets.HBox(children=[outw, q1]))

# plot into widget
with outw:
    h = show(plot, notebook_handle=True)


In [ ]:
from ipyaladin import Aladin

from ipywidgets import Layout, Box, widgets

aladin = Aladin(layout=Layout(width='70%'), target='Berkeley 20', fov=0.3)
info = widgets.HTML()


box_layout = Layout(display='flex',
                    flex_flow='row',
                    align_items='stretch',
                    width='100%')
box = Box(children=[aladin, info], layout=box_layout)
box

In [ ]:
import requests
import logging
data_list = []
def process_result(data):
    info.value = ''
    ra = data['ra']
    dec = data['dec']
    radius = min(aladin.fov / 10, 5)
    logging.warn(ra)
    logging.warn(dec)
    query = """SELECT TOP 1 main_id, ra, dec, DISTANCE(POINT('ICRS', %f, %f), POINT('ICRS', ra, dec)) as d FROM basic
               WHERE CONTAINS(POINT('ICRS', ra, dec), CIRCLE('ICRS', %f, %f, %f))=1
               ORDER BY d ASC""" % (ra, dec, ra, dec, aladin.fov / 10)
    
    r = requests.get('http://simbad.u-strasbg.fr/simbad/sim-tap/sync', params={'query': query, 'request': 'doQuery', 'lang': 'adql', 'format': 'json', 'phase': 'run'})
    obj_name = ''
    obj_coo = None
    obj_data = r.json()['data']
    if len(obj_data)==0:
        return
    
    obj_name = obj_data[0][0]
    obj_coo = [obj_data[0][1], obj_data[0][2]]
    data_list.append(obj_data)
    sed_img = '<img src="http://cdsportal.u-strasbg.fr/cgi-bin/PhotVizPreview/plot?ra=%f&dec=%f&radius_arcsec=5&w=200&h=150&point_size=4">' % (obj_coo[0], obj_coo[1])
    info.value =  '<h2>%s</h2><br><br>%s' % (obj_name, sed_img)
    
aladin.add_listener('click', process_result)


In [ ]:
source.data = {'x':[42], 'y':[42]}
push_notebook(handle=h)
h.comms

In [ ]:
aladin.add_table??

In [ ]:
import astropy
import healpy

In [ ]:
import numpy as np
import healpy as hp
from math import pi
def IndexToDeclRa(NSIDE, index):
    theta, phi = hp.pixelfunc.pix2ang(NSIDE,index)
    return -np.degrees(theta-pi/2.),np.degrees(math.pi*2.-phi)
IndexToDeclRa(9, 1464772)

In [ ]:
import hr
import bokeh
from bokeh.io import output_notebook, show, push_notebook
output_notebook(bokeh.resources.INLINE)
hr.visual.hr_diagram_selection('berkeley20')